# NLP with spaCy

*Based off a notebook by Alison Parrish*

## Installing spaCy

From the terminaly, type:

    sudo pip install spacy
    
(If you're one of the people having trouble with sudo, just try `pip install spacy`.)

Currently, spaCy is distributed in source form only, so the installation process involves a bit of compiling. On macOS, you'll need to install [XCode](https://developer.apple.com/xcode/) if you don't have it already in order to perform the compilation steps. [Here's a good tutorial for macOS Sierra](http://railsapps.github.io/xcode-command-line-tools.html), though the steps should be similar on other versions.

## Downloading the spaCy data ##

After you've installed spaCy, you'll need to download the data. Run the following on the command line:

    sudo python -m spacy download en
    
As above, if you're having trouble with sudo on your machine, just remove the "sudo" from the line above.

## Basic usage

Import `spacy` like any other Python module. The `spaCy` code expects all strings to be unicode strings, so make sure you've included `from __future__ import unicode_literals` at the top of your notebook—it'll make your life easier, trust me.

In [ ]:
from __future__ import unicode_literals
import spacy

Create a new spaCy object using `spacy.load('en')` (assuming you want to work with English; spaCy supports other languages as well).

In [ ]:
nlp = spacy.load('en')

# NOTE: If you are one of the people for whom sudo does not work, you will need to specify
# the longer name of the English language model as do:
# nlp = spacy.load('en_core_web_sm')

And then create a `Document` object by calling the spaCy object with the text you want to work with. Below I've included a few sentences from the Universal Declaration of Human Rights:

In [ ]:
doc = nlp("All human beings are born free and equal in dignity and rights. They are endowed with reason and conscience and should act towards one another in a spirit of brotherhood. Everyone has the right to life, liberty and security of person.")

## Sentences

If you learn nothing else about spaCy (or NLP), then learn at least that it's an easy way to get a list of sentences in a text. Once you've created a document object, you can iterate over the sentences it contains using the `.sents` attribute:

In [ ]:
for item in doc.sents:
    print(item.text)

Note: The `.sents` attribute is a generator, so you can't index or count it directly. To index or count the .sents attribute, you'll need to convert it to a list first using the `list()` function:

In [ ]:
sentences_as_list = list(doc.sents)

In [ ]:
# check the length to make sure it worked

len(sentences_as_list)

## Words

Iterating over a document yields each word in the document in turn. Words are represented with spaCy [Token](https://spacy.io/docs/api/token) objects, which have several interesting attributes. 

The `.text` attribute gives the underlying text of the word, and the `.lemma_` attribute gives the word's "lemma." 

Let's take a look:

In [ ]:
print("Word, lemma\n")
for word in doc:
    print(word.text + ", " + word.lemma_)
    
# Note: On the underscore at the end of a variable, see
# https://www.datacamp.com/community/tutorials/role-underscore-python


Individual sentences can also be iterated over to get a list of words:

In [ ]:
sentence = list(doc.sents)[1]  # same as sentence = sentences_as_list[1]

for word in sentence:
    print(word.text, word.lemma_)

## Parts of speech

The `pos_` attribute gives a general part of speech; the `tag_` attribute gives a more specific designation. [List of meanings here.](https://spacy.io/api/annotation#pos-tagging)

In [ ]:
print("Word, POS, tag\n")

for item in doc:
    print(item.text, item.pos_, item.tag_)

### Extracting words by part of speech

With knowledge of which part of speech each word belongs to, we can make simple code to extract and recombine words by their part of speech. The following code creates a list of all nouns and adjectives in the text:

In [ ]:
nouns = []
adjectives = []
for item in doc:
    if item.pos_ == 'NOUN':
        nouns.append(item.text)
for item in doc:
    if item.pos_ == 'ADJ':
        adjectives.append(item.text)

And below, some code to print out random pairings of an adjective from the text with a noun from the text:

In [ ]:
print(random.choice(adjectives) + " " + random.choice(nouns))

Making a list of verbs works similarly:

In [ ]:
verbs = []
for item in doc:
    if item.pos_ == 'VERB':
        verbs.append(item.text)

Although in this case, you'll notice the list of verbs is a bit unintuitive. We're getting words like "should" and "are" and "has"—helper verbs that maybe don't fit our idea of what verbs we want to extract.

In [ ]:
verbs

This is because we used the `.pos_` attribute, which only gives us general information about the part of speech. The `.tag_` attribute allows us to be more specific about the kinds of verbs we want. 

**HOW WOULD WE GET ONLY VERBS IN PAST PARTICPLE?**

In [ ]:
only_past = []

for item in doc:
    # THEN WHAT???
    
    
    
    
    
only_past

## Larger syntactic units

So we can get individual words by their part of speech. Great! But what if we want larger chunks, based on their syntactic role in the sentence? The easy way is `.noun_chunks`, which is an attribute of a document or a sentence that evaluates to a list of [spans](https://spacy.io/docs/api/span) of noun phrases, regardless of their position in the document:

In [ ]:
for item in doc.noun_chunks:
    print(item.text)

For anything more sophisticated than this, though, we'll need to learn about how spaCy parses sentences into its syntactic components.

### Understanding dependency grammars

![displacy parse](http://static.decontextualize.com/syntax_example.png)

The idea of a dependency grammar is that every word in a sentence is a "dependent" of some other word, which is that word's "head." Those "head" words are in turn dependents of other words. The finite verb in the sentence is the ultimate "head" of the sentence, and is not itself dependent on any other word. (The dependents of a particular head are sometimes called its "children.")

The question of how to know what constitutes a "head" and a "dependent" is complicated. As a starting point, here's a passage from [Dependency Grammar and Dependency Parsing](http://stp.lingfil.uu.se/~nivre/docs/05133.pdf):

> Here are some of the criteria that have been proposed for identifying a syntactic relation between a head H and a dependent D in a construction C (Zwicky, 1985; Hudson, 1990):
>
> 1. H determines the syntactic category of C and can often replace C.
> 2. H determines the semantic category of C; D gives semantic specification.
> 3. H is obligatory; D may be optional.
> 4. H selects D and determines whether D is obligatory or optional.
> 5. The form of D depends on H (agreement or government).
> 6. The linear position of D is specified with reference to H."

There are different *types* of relationships between heads and dependents, and each type of relation has its own name. 

**Use [the displaCy visualizer from earlier](https://demos.explosion.ai/displacy/?text=Everyone%20has%20the%20right%20to%20life%2C%20liberty%20and%20security%20of%20person&model=en&cpu=1&cph=0) to see how a particular sentence is parsed, and what the relations between the heads and dependents are.**

Here's a list of a few dependency relations and what they mean. ([A more complete list can be found here.](http://www.mathcs.emory.edu/~choi/doc/clear-dependency-2012.pdf))

* `nsubj`: this word's head is a verb, and this word is itself the subject of the verb
* `nsubjpass`: same as above, but for subjects in sentences in the passive voice
* `dobj`: this word's head is a verb, and this word is itself the direct object of the verb
* `iobj`: same as above, but indirect object
* `aux`: this word's head is a verb, and this word is an "auxiliary" verb (like "have", "will", "be")
* `attr`: this word's head is a copula (like "to be"), and this is the description attributed to the subject of the sentence (e.g., in "This product is a global brand", `brand` is dependent on `is` with the `attr` dependency relation)
* `det`: this word's head is a noun, and this word is a determiner of that noun (like "the," "this," etc.)
* `amod`: this word's head is a noun, and this word is an adjective describing that noun
* `prep`: this word is a preposition that modifies its head
* `pobj`: this word is a dependent (object) of a preposition

In [ ]:
# Let's take a look at how this works in practice

for word in list(doc.sents)[2]:
    print("Word:", word.text)
    print("Tag:", word.tag_)
    print("Head:", word.head.text)
    print("Dependency relation:", word.dep_)
    print("Children:", list(word.children))
    print("")

### Using .subtree for extracting syntactic units

The `.subtree` attribute evaluates to a generator that can be flatted by passing it to `list()`. This is a list of the word's syntactic dependents—essentially, the "clause" that the word belongs to.

This function merges a subtree and returns a string with the text of the words contained in it:

In [ ]:
def flatten_subtree(st):
       return ''.join([w.text_with_ws for w in list(st)]).strip() # just take my word for it!

With this function in our toolbox, we can write a loop that prints out the subtree for each word in a sentence:

In [ ]:
for word in list(doc.sents)[2]:
    print("Word:", word.text)
    print("Flattened subtree: ", flatten_subtree(word.subtree))
    print("")

Using the subtree and our knowledge of dependency relation types, we can write code that extracts larger syntactic units based on their relationship with the rest of the sentence. For example, to get all of the noun phrases that are subjects of a verb:

In [ ]:
subjects = []
for word in doc:
    if word.dep_ in ('nsubj', 'nsubjpass'):
        subjects.append(flatten_subtree(word.subtree))

In [ ]:
subjects

Or every prepositional phrase:

In [ ]:
prep_phrases = []
for word in doc:
    if word.dep_ == 'prep':
        prep_phrases.append(flatten_subtree(word.subtree))

In [ ]:
prep_phrases

## Entity extraction

A common task in NLP is taking a text and extracting "named entities" from it—basically, proper nouns, or names of companies, products, locations, etc. You can easily access this information using the `.ents` property of a document.

In [ ]:
doc2 = nlp("Claire Sterk and I visited the Apple Store in Decatur.")

In [ ]:
for item in doc2.ents:
    print(item)

Entity objects have a `.label_` attribute that tells you the type of the entity. ([Here's a full list of the built-in entity types.](https://spacy.io/docs/usage/entity-recognition#entity-types))

In [ ]:
for item in doc2.ents:
    print(item.text, item.label_)

[More on spaCy entity recognition.](https://spacy.io/docs/usage/entity-recognition)

## Loading data from a file

You can load data from a file easily with spaCy. You just have to make sure that the data is in Unicode format, not plain-text. An easy way to do this is to call `.decode('utf8')` on the string after you've loaded it:

In [ ]:
filename = "./2019-09-ccp-corpus-0.3/ccprecords/1850.ME-10.20.PORT.ART.01.txt"

with open(filename, "r", encoding="utf-8") as file:
    text = file.read()
    doc3 = nlp(text) # remember to convert it to a spacy object

And lo and behold, the named entities!

In [ ]:
for item in doc3.ents:
    print(item.text, item.label_)

## Further reading and resources

[A few example programs can be found here.](https://github.com/aparrish/rwet-examples/tree/master/spacy)

We've barely scratched the surface of what it's possible to do with spaCy. [There's a great page of tutorials on the official site](https://spacy.io/docs/usage/tutorials) that you should check out!